# Project: Best place to open a Vietnamese restaurant in Toronto
##  Introduction/Business Problem 
A new businessman who has just arrived to Toronto, Ontario, Canada and wants to open a Vietnamese restaurant. However, there are too many Vietnamese restaurants, as well as Asian restaurants in Toronto. This project will find out the best neighborhood in Toronto to open a Vietnamese restaurant.
The best locations will have no or very few Vietnamese restaurants, and have many non-Vietnamese restaurants to guarantee amount of customers.
## Data
- Data on Vietnamese restaurants and other restaurants on each neighborhood will be obtained using foursquare API
- K-mean will be used to create zones of good neighborhoods for opening the restaurants

In [32]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import json
import requests

### Extract data using pandas

In [33]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(page)[0]
table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### drop rows that have Borough='Not assigned'

In [34]:
df = table.drop(table[table.Borough=='Not assigned'].index)

In [35]:
df = df.reset_index(drop=True)
df = df.rename(columns = {'Neighbourhood': 'Neighborhood'})

In [36]:
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## read the latitude and longitude into data frame

In [37]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [38]:
geo.shape

(103, 3)

In [39]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## merge two dataframes based on postal code

In [40]:
neighborhoods = pd.merge(df, geo, on='Postal Code')

In [41]:
neighborhoods.shape

(103, 5)

In [42]:
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# now let's explore neighborhoods in Toronto

In [43]:
neighborhoods['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [44]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [45]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


## Create a map of Toronto with neighborhoods superimposed on top

In [46]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Enter client ID and secret

In [47]:
CLIENT_ID = 'DOAZABZVLAW44SC5CVE3MNQUVYM3K5DTSNWTN5ZGZQXAD5SA' # your Foursquare ID
CLIENT_SECRET = 'W0B4YMT2IFD0K2SORXENNLYCJYDMD201RXXRKPS1PUZSCXYD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DOAZABZVLAW44SC5CVE3MNQUVYM3K5DTSNWTN5ZGZQXAD5SA
CLIENT_SECRET:W0B4YMT2IFD0K2SORXENNLYCJYDMD201RXXRKPS1PUZSCXYD


In [75]:
neighborhood_latitude = downtown.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


### Now, let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 1 km.

In [26]:
results = requests.get(url).json()
results

NameError: name 'url' is not defined

### write a function that extract the category of the venue

In [49]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### extract all information in items key

In [50]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

NameError: name 'results' is not defined

In [85]:
nearby_venues['categories'].unique()

array(['Bakery', 'Coffee Shop', 'Restaurant', 'Historic Site', 'Park',
       'Farmers Market', 'Distribution Center',
       'Mediterranean Restaurant', 'Dessert Shop', 'Pub',
       'Chocolate Shop', 'Performing Arts Venue', 'French Restaurant',
       'Liquor Store', 'Spa', 'Breakfast Spot', 'Tech Startup',
       'Italian Restaurant', 'Café', 'Mexican Restaurant',
       'Gym / Fitness Center', 'Thai Restaurant', 'Yoga Studio',
       'Shoe Store', 'Pool', 'Theater', 'Animal Shelter',
       'Asian Restaurant', 'Furniture / Home Store', 'Event Space',
       'Ice Cream Shop', 'Gastropub', 'German Restaurant', 'Diner',
       'Sandwich Place', 'Grocery Store', 'Art Gallery', 'Food Truck',
       'Pizza Place', 'Middle Eastern Restaurant', 'Gym Pool', 'Bar',
       'Athletics & Sports', 'Electronics Store', 'Dance Studio',
       'Burrito Place', 'Karaoke Bar', 'Brewery', 'Sushi Restaurant',
       'Auto Dealership', 'Bank', 'Pet Store', 'Beer Store',
       'Skating Rink', 'Pharmacy

In [86]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


###  create a function to get all venues of all the neighborhoods in Toronto

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### run the above function on each neighborhood and create a new dataframe called toronto_venues

In [52]:
# type your answer here

toronto_venue = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [53]:
toronto_venue.shape

(4905, 7)

In [54]:
toronto_venue.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


In [142]:
toronto_venue['Venue Category'].unique()

array(['Caribbean Restaurant', 'Park', 'Café', 'Fast Food Restaurant',
       'Grocery Store', 'Pharmacy', 'Supermarket', 'Fish & Chips Shop',
       'Pizza Place', 'Food & Drink Shop', 'Road', 'Bus Stop',
       'Train Station', 'Discount Store', 'Laundry Service',
       'Chinese Restaurant', 'Coffee Shop', 'Convenience Store',
       'Shopping Mall', 'Tennis Court', 'Cosmetics Shop',
       'Shop & Service', 'Hockey Arena', 'Portuguese Restaurant',
       'Golf Course', 'French Restaurant', 'Intersection', "Men's Store",
       'Boxing Gym', 'Gym / Fitness Center', 'Bakery', 'Restaurant',
       'Historic Site', 'Farmers Market', 'Distribution Center',
       'Mediterranean Restaurant', 'Dessert Shop', 'Pub',
       'Chocolate Shop', 'Performing Arts Venue', 'Liquor Store', 'Spa',
       'Breakfast Spot', 'Tech Startup', 'Italian Restaurant',
       'Mexican Restaurant', 'Thai Restaurant', 'Yoga Studio',
       'Shoe Store', 'Pool', 'Theater', 'Animal Shelter',
       'Asian Restaur

### only select venue as restaurant

In [55]:
toronto_restaurants = toronto_venue[toronto_venue['Venue Category'].str.contains('Restaurant')]

In [56]:
toronto_restaurants = toronto_restaurants.reset_index(drop=True)

In [57]:
toronto_restaurants.shape

(1163, 7)

In [58]:
toronto_restaurants['Venue'].unique()

array(["Allwyn's Bakery", 'A&W', 'Spicy Chicken House', 'Portugril',
       'The Frig', 'Impact Kitchen', 'Souk Tabule',
       'Cluny Bistro & Boulangerie', 'Mangia and Bevi Resto-Bar',
       'El Catrin', 'Sukhothai', "Fusaro's", 'Schnitzel Queen',
       'Cluck Clucks', 'Paintbox Bistro', 'Mystic Muffin', 'Qi sushi',
       'Sushi Bar', 'Sultan Of Samosas', "King's Place647-352-0786",
       'Lac Vien Vietnamese Restaurant', 'Yuki Japanese Restaurant',
       "The Burger's Priest", 'Krystos', "Harvey's", 'Tora Aburi Sushi',
       'Red Lobster', 'Burger King', 'Viet House', "Shoeless Joe's",
       'Wendy’s', 'Tokyo Sushi', 'Como En Casa', 'Sansotei Ramen 三草亭',
       "Somethin' 2 Talk About", 'Vegetarian Haven', 'Ethiopian House',
       'Scaddabush Italian Kitchen & Bar', 'Storm Crow Manor',
       'The Elm Tree Restaurant', 'Kothur Indian Cuisine', 'Wish',
       'Si Lom', 'KAKA', 'Kinka Izakaya Original', 'Fabarnak', 'Japango',
       'Paramount Fine Foods', 'Trattoria Nervosa',

In [59]:
toronto_restaurants[toronto_restaurants['Venue Category'].str.contains('Vietnamese')].count()

Neighborhood              30
Neighborhood Latitude     30
Neighborhood Longitude    30
Venue                     30
Venue Latitude            30
Venue Longitude           30
Venue Category            30
dtype: int64

In [60]:
vietnamese_restaurants = toronto_restaurants[toronto_restaurants['Venue Category'].str.contains('Vietnamese')]

In [61]:
vietnamese_restaurants = vietnamese_restaurants.reset_index(drop=True)

In [121]:
vietnamese_restaurants.shape

(30, 7)

### check how many venues were returned for each neighborhood

In [63]:
toronto_restaurants.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,19,19,19,19,19,19
"Alderwood, Long Branch",1,1,1,1,1,1
"Bathurst Manor, Wilson Heights, Downsview North",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",15,15,15,15,15,15
...,...,...,...,...,...,...
"Willowdale, Willowdale East",36,36,36,36,36,36
"Willowdale, Willowdale West",1,1,1,1,1,1
Woburn,3,3,3,3,3,3


### display vietnamese restaurants and toronto neighborhoods on the map
red = Vietnamese restaurant
blue = Toronto neighborhoods

In [64]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    

# add markers to map
for lat, lng, venue, neighborhood in zip(vietnamese_restaurants['Venue Latitude'], vietnamese_restaurants['Venue Longitude'], vietnamese_restaurants['Venue'], vietnamese_restaurants['Neighborhood']):
    label = '{}, {}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Converting all Venue Category into either Vietnamese Restaurant or Non-Vietnamese Restaurant

In [91]:
toronto_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Spicy Chicken House,43.760639,-79.325671,Chinese Restaurant
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant


In [97]:
df1 = toronto_restaurants.groupby(by='Neighborhood', as_index=False).agg({'Venue Category': pd.Series.nunique})

In [98]:
df1.head()

,Neighborhood,Venue Category
0,Agincourt,13
1,"Alderwood, Long Branch",1
2,"Bathurst Manor, Wilson Heights, Downsview North",4
3,Bayview Village,3
4,"Bedford Park, Lawrence Manor East",10


In [102]:
df1 = df1.sort_values(by=['Venue Category'], ascending=False)

In [105]:
top_neighborhood = df1.head(10)

In [107]:
top_neighborhood = top_neighborhood.reset_index(drop=True)

In [108]:
top_neighborhood

,Neighborhood,Venue Category
0,"Little Portugal, Trinity",18
1,Davisville,18
2,"University of Toronto, Harbord",18
3,Christie,17
4,"The Annex, North Midtown, Yorkville",16
5,"The Danforth West, Riverdale",16
6,"Kensington Market, Chinatown, Grange Park",15
7,"East Toronto, Broadview North (Old East York)",15
8,Studio District,15
9,"Willowdale, Willowdale East",14


In [92]:
toronto_restaurants.groupby('Neighborhood').mean().reset_index()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude
0,Agincourt,43.794200,-79.262029,43.790684,-79.264844
1,"Alderwood, Long Branch",43.602414,-79.543484,43.609938,-79.549020
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,43.754598,-79.444207
3,Bayview Village,43.786947,-79.385975,43.789893,-79.386904
4,"Bedford Park, Lawrence Manor East",43.733282,-79.419750,43.733094,-79.419416
...,...,...,...,...,...
85,"Willowdale, Willowdale East",43.770120,-79.408493,43.769288,-79.411771
86,"Willowdale, Willowdale West",43.782736,-79.442259,43.774325,-79.442651
87,Woburn,43.770992,-79.216917,43.764031,-79.219139
88,Woodbine Heights,43.695344,-79.318389,43.696180,-79.328958


In [93]:
toronto_restaurants_merge = toronto_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Spicy Chicken House,43.760639,-79.325671,Chinese Restaurant
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant


In [81]:
restaurant_convert = toronto_restaurants.copy()

In [82]:
for i in restaurant_convert.index:
    if restaurant_convert.at[i,'Venue Category'] != 'Vietnamese Restaurant':
        restaurant_convert.at[i,'Venue Category'] = 'Non-Vietnamese Restaurant'
        

In [83]:
restaurant_convert['Venue Category'].value_counts()

Non-Vietnamese Restaurant    1133
Vietnamese Restaurant          30
Name: Venue Category, dtype: int64

In [109]:
restaurant_convert.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Non-Vietnamese Restaurant
1,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Non-Vietnamese Restaurant
2,Parkwoods,43.753259,-79.329656,Spicy Chicken House,43.760639,-79.325671,Non-Vietnamese Restaurant
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Non-Vietnamese Restaurant
4,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,Non-Vietnamese Restaurant


## Analyze Each Neighborhood

In [84]:
# one hot encoding
toronto_onehot = pd.get_dummies(restaurant_convert[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = restaurant_convert['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_onehot.shape

(1163, 3)

In [85]:
toronto_onehot.head()

,Neighborhood,Non-Vietnamese Restaurant,Vietnamese Restaurant
0,Parkwoods,1,0
1,Parkwoods,1,0
2,Parkwoods,1,0
3,Victoria Village,1,0
4,Victoria Village,1,0


In [67]:
toronto_onehot.groupby('Neighborhood')

### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [86]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Non-Vietnamese Restaurant,Vietnamese Restaurant
0,Agincourt,0.947368,0.052632
1,"Alderwood, Long Branch",1.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",1.000000,0.000000
3,Bayview Village,1.000000,0.000000
4,"Bedford Park, Lawrence Manor East",1.000000,0.000000
...,...,...,...
85,"Willowdale, Willowdale East",0.972222,0.027778
86,"Willowdale, Willowdale West",1.000000,0.000000
87,Woburn,1.000000,0.000000
88,Woodbine Heights,1.000000,0.000000


In [110]:
result = pd.merge(top_neighborhood,
                 toronto_grouped[['Neighborhood', 'Vietnamese Restaurant']],
                 on='Neighborhood')

In [116]:
result

,Neighborhood,Venue Category,Vietnamese Restaurant
8,Studio District,15,14.285714
6,"Kensington Market, Chinatown, Grange Park",15,8.333333
0,"Little Portugal, Trinity",18,6.451613
2,"University of Toronto, Harbord",18,3.703704
3,Christie,17,2.941176
9,"Willowdale, Willowdale East",14,2.777778
1,Davisville,18,2.564103
4,"The Annex, North Midtown, Yorkville",16,0.000000
5,"The Danforth West, Riverdale",16,0.000000
7,"East Toronto, Broadview North (Old East York)",15,0.000000


In [112]:
result['Vietnamese Restaurant'] = result['Vietnamese Restaurant'] * 100

In [118]:
result = result.sort_values(by='Vietnamese Restaurant')

# list of neighborhoods with most restaurants and least Vietnamese restaurants

In [119]:
result

,Neighborhood,Venue Category,Vietnamese Restaurant
4,"The Annex, North Midtown, Yorkville",16,0.000000
5,"The Danforth West, Riverdale",16,0.000000
7,"East Toronto, Broadview North (Old East York)",15,0.000000
1,Davisville,18,2.564103
9,"Willowdale, Willowdale East",14,2.777778
3,Christie,17,2.941176
2,"University of Toronto, Harbord",18,3.703704
0,"Little Portugal, Trinity",18,6.451613
6,"Kensington Market, Chinatown, Grange Park",15,8.333333
8,Studio District,15,14.285714
